In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys

In [2]:
SCC = pd.read_csv("../../data/Graph_1_5/SCC.csv")
Louv = pd.read_csv("../../data/Graph_1_5/Louv.csv")
LabelProp = pd.read_csv("../../data/Graph_1_5/LabelProp.csv")
Leiden = pd.read_csv("../../data/Graph_1_5/Leiden.csv")
nodes = pd.read_csv("../../data/Graph_1_5/nodes.csv")
Between = pd.read_csv("../../data/Graph_1_5/Betweeness.csv")
PageRank = pd.read_csv("../../data/Graph_1_5/PageRank.csv")
Closeness = pd.read_csv("../../data/Graph_1_5/closeness.csv")

In [3]:
def community_det_acc(df1, df2):
    df1['Node_ID:ID'] = [int(i.strip('"')) for i in df1['Name'].tolist()]
    df1.drop(columns=['Name'], inplace=True)

    result_df = pd.merge(df1, df2, on='Node_ID:ID', how='inner')

    percentage_breakdown = result_df.groupby(['Community', 'Status:LABEL']).size().unstack(fill_value=0)
    
    percentage_breakdown_percentage = percentage_breakdown.div(percentage_breakdown.sum(axis=1), axis=0) * 100
    
    return percentage_breakdown_percentage

In [4]:
community_det_acc(SCC, nodes)

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
0,33.248599,39.368729,27.382672


In [6]:
community_det_acc(Louv, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
269,28.461538,38.076923,33.461538
289,18.891688,39.294710,41.813602
341,33.606557,32.377049,34.016393
355,36.790123,16.543210,46.666667
405,40.947368,39.473684,19.578947


In [5]:
community_det_acc(LabelProp, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
1,28.557030,19.603561,51.839409
6,26.388069,31.069108,42.542823
28,34.192099,38.182058,27.625842
66,15.248227,35.106383,49.645390
78,7.936508,50.793651,41.269841


In [6]:
community_det_acc(Leiden, nodes).head()

Status:LABEL,Hyperglycemia,Hypoglycemia,Normal
Community,,,
2,39.507620,40.767878,19.724502
3,27.090153,56.684066,16.225781
7,31.702128,54.158607,14.139265
12,32.394366,17.706237,49.899396
14,99.894180,0.105820,0.000000


In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from scipy.signal import find_peaks
from scipy.signal import peak_widths
from scipy.signal import peak_prominences
from scipy.integrate import simps
from sklearn.model_selection import StratifiedKFold

In [5]:
def clean_merge(Louv, nodes):
    Louv['Node_ID:ID'] = [int(i.strip('"')) for i in Louv['Name'].tolist()]
    Louv.drop(columns=['Name'], inplace=True)
    return pd.merge(Louv, nodes, on='Node_ID:ID', how='inner')

In [ ]:
Between['Node_ID:ID'] = [int(i.strip('"')) for i in Between['Name'].tolist()]
Between.drop(columns=['Name'], inplace=True)

In [32]:
result_df = pd.merge(Between, result_df, on='Node_ID:ID', how='inner')

NameError: name 'result_df' is not defined

In [7]:
Leiden.rename(columns={'Community':'Leiden'}, inplace=True)
LabelProp.rename(columns={'Community':'LabelProp'}, inplace=True)
Louv.rename(columns={'Community':'Louv'}, inplace=True)
PageRank.rename(columns={'score':'PageRank'}, inplace=True)
Between.rename(columns={'score':'Between'}, inplace=True)

In [8]:
Closeness.rename(columns={'score':'Closeness'}, inplace=True)

In [9]:
result_df = clean_merge(Leiden, nodes)
result_df = clean_merge(LabelProp, result_df)
result_df = clean_merge(Louv, result_df)
result_df = clean_merge(Between, result_df)
result_df = clean_merge(PageRank, result_df)
result_df = clean_merge(Closeness, result_df)

In [8]:
Closeness.rename(columns={'nodeId':'Name'}, inplace=True)

In [9]:
Closeness.rename(columns={'Name':'Node_ID:ID'}, inplace=True)

In [10]:
result_df = pd.merge(Closeness, result_df, on='Node_ID:ID', how='inner')

ValueError: You are trying to merge on object and int64 columns for key 'Node_ID:ID'. If you wish to proceed you should use pd.concat

In [10]:
Closeness

,Closeness,Node_ID:ID
0,0.342238,5167722
1,0.340816,5168623
2,0.340037,5168215
3,0.339736,5197403
4,0.339343,5196706
...,...,...
51922,0.129579,1197193
51923,0.129579,1197218
51924,0.129579,1197283
51925,0.129579,1197368


In [11]:
result_df

,Closeness,Node_ID:ID,PageRank,Between,Louv,LabelProp,Leiden,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot
0,0.342238,5167722,0.932545,1.812212e+07,7870,4148,3,210407-1-02,532.80182,7956.8179,Hypoglycemia,11.514477,5510.6140,"(10, 7)"
1,0.340816,5168623,0.788729,1.639455e+07,7870,4148,3,210407-1-02,1148.51940,9168.3438,Hypoglycemia,22.834515,7484.9468,"(22, 9)"
2,0.340037,5168215,1.135755,4.688332e+06,18505,4148,7,210407-1-02,869.70386,4263.9268,Hypoglycemia,17.319799,2687.4705,"(17, 4)"
3,0.339736,5197403,1.207448,6.900229e+06,46582,6,410,210407-1-03,1008.42820,2218.3857,Hypoglycemia,10.095751,177.0571,"(20, 2)"
4,0.339343,5196706,0.911400,2.251702e+07,7870,826,3,210407-1-03,532.11847,9018.3037,Hypoglycemia,9.251050,3260.3818,"(10, 9)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51922,0.129579,1197193,0.943612,0.000000e+00,14998,14839,270,210211-2-05,817.08429,34853.7660,Hyperglycemia,10.947153,2244.9710,"(16, 34)"
51923,0.129579,1197218,0.943612,0.000000e+00,14998,14839,270,210211-2-05,834.16858,35904.6800,Hyperglycemia,77.462750,10884.4200,"(16, 35)"
51924,0.129579,1197283,0.943612,0.000000e+00,14998,14839,270,210211-2-05,878.58771,28607.2050,Hyperglycemia,13.379180,882.1950,"(17, 28)"
51925,0.129579,1197368,0.962131,0.000000e+00,14998,14839,270,210211-2-05,936.67426,27065.9940,Hyperglycemia,9.285264,395.6700,"(18, 27)"


In [12]:
result_df['GridSlot_x'] = 0
result_df['GridSlot_y'] = 0

for i, r in result_df.iterrows():
    result_df['GridSlot_x'][i], result_df['GridSlot_y'][i] = r['GridSlot'].split(',')
    result_df['GridSlot_x'][i] = int(result_df['GridSlot_x'][i].strip('('))
    result_df['GridSlot_y'][i] = int(result_df['GridSlot_y'][i].strip(')'))

C:\Users\stang\AppData\Local\Temp\ipykernel_13812\1595376135.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['GridSlot_x'][i], result_df['GridSlot_y'][i] = r['GridSlot'].split(',')
C:\Users\stang\AppData\Local\Temp\ipykernel_13812\1595376135.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '(10' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result_df['GridSlot_x'][i], result_df['GridSlot_y'][i] = r['GridSlot'].split(',')
C:\Users\stang\AppData\Local\Temp\ipykernel_13812\1595376135.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [13]:
result_df

,Closeness,Node_ID:ID,PageRank,Between,Louv,LabelProp,Leiden,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot,GridSlot_x,GridSlot_y
0,0.342238,5167722,0.932545,1.812212e+07,7870,4148,3,210407-1-02,532.80182,7956.8179,Hypoglycemia,11.514477,5510.6140,"(10, 7)",10,7
1,0.340816,5168623,0.788729,1.639455e+07,7870,4148,3,210407-1-02,1148.51940,9168.3438,Hypoglycemia,22.834515,7484.9468,"(22, 9)",22,9
2,0.340037,5168215,1.135755,4.688332e+06,18505,4148,7,210407-1-02,869.70386,4263.9268,Hypoglycemia,17.319799,2687.4705,"(17, 4)",17,4
3,0.339736,5197403,1.207448,6.900229e+06,46582,6,410,210407-1-03,1008.42820,2218.3857,Hypoglycemia,10.095751,177.0571,"(20, 2)",20,2
4,0.339343,5196706,0.911400,2.251702e+07,7870,826,3,210407-1-03,532.11847,9018.3037,Hypoglycemia,9.251050,3260.3818,"(10, 9)",10,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51922,0.129579,1197193,0.943612,0.000000e+00,14998,14839,270,210211-2-05,817.08429,34853.7660,Hyperglycemia,10.947153,2244.9710,"(16, 34)",16,34
51923,0.129579,1197218,0.943612,0.000000e+00,14998,14839,270,210211-2-05,834.16858,35904.6800,Hyperglycemia,77.462750,10884.4200,"(16, 35)",16,35
51924,0.129579,1197283,0.943612,0.000000e+00,14998,14839,270,210211-2-05,878.58771,28607.2050,Hyperglycemia,13.379180,882.1950,"(17, 28)",17,28
51925,0.129579,1197368,0.962131,0.000000e+00,14998,14839,270,210211-2-05,936.67426,27065.9940,Hyperglycemia,9.285264,395.6700,"(18, 27)",18,27


In [14]:
result_df.drop(columns={"GridSlot"}, inplace=True)

In [15]:
peaks_df = result_df

In [16]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame and 'column_name' is the column you want to normalize
column_name = 'PageRank'

scaler = MinMaxScaler()
peaks_df[[column_name]] = scaler.fit_transform(peaks_df[[column_name]])

In [17]:
column_name = 'Widths:float'

scaler = MinMaxScaler()
peaks_df[[column_name]] = scaler.fit_transform(peaks_df[[column_name]])

In [18]:
# Define a function to calculate the bin for a given wavenumber with a specified bin size
def calculate_bin_interval(wavenumber, bin_size):
    bin_start = int((wavenumber - 200) / bin_size) * bin_size + 200
    bin_end = bin_start + bin_size
    return f"{bin_start}-{bin_end}"

# Set the bin size
bin_size = 100

# Add a "Bin" column to the DataFrame
peaks_df['Bin'] = peaks_df['WaveNumber:float'].apply(lambda x: calculate_bin_interval(x, bin_size))

In [159]:
# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = peaks_df.pivot_table(index='SpecID', columns='Bin', values=['Widths:float', 'Prominences:float', 'GridSlot_y'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = peaks_df[['SpecID', 'Status:LABEL']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with False
peak_bins.fillna(False, inplace=True)

C:\Users\stang\AppData\Local\Temp\ipykernel_13812\831350103.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  peak_bins.fillna(False, inplace=True)


In [160]:
# Splitting the dataframe into features (X) and target variable (y)
X = peak_bins.drop(['Status:LABEL'], axis=1)
y = peak_bins['Status:LABEL']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [127]:
X.head()

,GridSlot_y_1000-1100,GridSlot_y_1100-1200,GridSlot_y_1200-1300,GridSlot_y_1300-1400,GridSlot_y_1400-1500,GridSlot_y_1500-1600,GridSlot_y_1600-1700,GridSlot_y_1700-1800,GridSlot_y_1800-1900,GridSlot_y_1900-2000,...,Widths:float_1800-1900,Widths:float_1900-2000,Widths:float_200-300,Widths:float_300-400,Widths:float_400-500,Widths:float_500-600,Widths:float_600-700,Widths:float_700-800,Widths:float_800-900,Widths:float_900-1000
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,False,False,False,False,False,1.0,False,False,False,False,...,False,False,False,False,False,False,0.027845,False,False,0.042921
201210-1-01,False,1.0,1.0,False,1.0,1.0,1.0,False,False,False,...,False,False,False,False,False,0.000557,0.023936,False,0.034161,0.010699
201210-1-02,2.0,2.0,2.0,2.0,2.0,False,2.0,2.0,False,False,...,False,False,False,False,0.000705,False,0.013644,False,False,0.014281
201210-1-03,2.0,3.0,2.0,False,3.0,2.0,2.0,False,False,False,...,False,False,0.00758,False,0.007039,False,0.018623,False,False,0.025602
201210-1-04,2.0,2.0,2.0,False,2.0,2.0,2.0,False,False,False,...,False,False,False,False,0.006819,False,0.023122,False,False,False


In [161]:
rf = RandomForestClassifier(random_state=1234)
rf.fit(X_train, y_train)

# Initialising and fitting the Extra Trees classifier
et = ExtraTreesClassifier(random_state=1234)
et.fit(X_train, y_train)

ExtraTreesClassifier(random_state=1234)

In [23]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(20)

    return top_10_features

In [24]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [162]:
# Making predictions on the test set
y_pred = rf.predict(X_test)

# Calculating and printing evaluation metrics
calculate_metrics(y_test, y_pred)

Overall Accuracy: 0.8221476510067114


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.81      0.82      0.81       195
 Hypoglycemia       0.83      0.81      0.82       196
       Normal       0.83      0.84      0.83       205

     accuracy                           0.82       596
    macro avg       0.82      0.82      0.82       596
 weighted avg       0.82      0.82      0.82       596


Confusion Matrix:
[[159  17  19]
 [ 20 159  17]
 [ 18  15 172]]


In [154]:
# Making predictions on the test set
y_pred = et.predict(X_test)

# Calculating and printing evaluation metrics
calculate_metrics(y_test, y_pred)

Overall Accuracy: 0.8053691275167785


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.80      0.82      0.81       195
 Hypoglycemia       0.83      0.79      0.81       196
       Normal       0.79      0.81      0.80       205

     accuracy                           0.81       596
    macro avg       0.81      0.81      0.81       596
 weighted avg       0.81      0.81      0.81       596


Confusion Matrix:
[[159  15  21]
 [ 19 154  23]
 [ 21  17 167]]


In [163]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

cv_results = cross_val_score(rf, X, y, cv=cv, scoring='accuracy')

In [164]:
print(cv_results)
print(f"Mean Accuracy: {cv_results.mean():.2f}")
print(f"Standard Deviation: {cv_results.std():.2f}")

[0.87248322 0.83557047 0.83892617 0.8557047  0.86241611 0.79530201
 0.83557047 0.80872483 0.82550336 0.83501684]
Mean Accuracy: 0.84
Standard Deviation: 0.02


In [123]:
get_feature_importances(rf, X)

,Feature,Importance
82,Widths:float_200-300,0.045209
64,Prominences:float_200-300,0.041467
88,Widths:float_800-900,0.026607
46,PageRank_200-300,0.025663
28,GridSlot_y_200-300,0.025142
70,Prominences:float_800-900,0.022972
54,Prominences:float_1000-1100,0.022701
74,Widths:float_1200-1300,0.022665
18,GridSlot_y_1000-1100,0.020778
72,Widths:float_1000-1100,0.020020


In [124]:
get_feature_importances(et, X)

,Feature,Importance
64,Prominences:float_200-300,0.027758
82,Widths:float_200-300,0.026165
28,GridSlot_y_200-300,0.024824
46,PageRank_200-300,0.021990
18,GridSlot_y_1000-1100,0.021242
51,PageRank_700-800,0.019537
75,Widths:float_1300-1400,0.018631
32,GridSlot_y_600-700,0.018051
30,GridSlot_y_400-500,0.017963
36,PageRank_1000-1100,0.017515


In [54]:
result_df

,Closeness,Node_ID:ID,PageRank,Between,Louv,LabelProp,Leiden,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot_x,GridSlot_y,Bin
0,0.342238,5167722,0.628102,1.812212e+07,7870,4148,3,210407-1-02,532.80182,7956.8179,Hypoglycemia,0.003874,5510.6140,10,7,500-600
1,0.340816,5168623,0.508633,1.639455e+07,7870,4148,3,210407-1-02,1148.51940,9168.3438,Hypoglycemia,0.011829,7484.9468,22,9,1100-1200
2,0.340037,5168215,0.796911,4.688332e+06,18505,4148,7,210407-1-02,869.70386,4263.9268,Hypoglycemia,0.007954,2687.4705,17,4,800-900
3,0.339736,5197403,0.856466,6.900229e+06,46582,6,410,210407-1-03,1008.42820,2218.3857,Hypoglycemia,0.002878,177.0571,20,2,1000-1100
4,0.339343,5196706,0.610537,2.251702e+07,7870,826,3,210407-1-03,532.11847,9018.3037,Hypoglycemia,0.002284,3260.3818,10,9,500-600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51922,0.129579,1197193,0.637295,0.000000e+00,14998,14839,270,210211-2-05,817.08429,34853.7660,Hyperglycemia,0.003476,2244.9710,16,34,800-900
51923,0.129579,1197218,0.637295,0.000000e+00,14998,14839,270,210211-2-05,834.16858,35904.6800,Hyperglycemia,0.050214,10884.4200,16,35,800-900
51924,0.129579,1197283,0.637295,0.000000e+00,14998,14839,270,210211-2-05,878.58771,28607.2050,Hyperglycemia,0.005185,882.1950,17,28,800-900
51925,0.129579,1197368,0.652680,0.000000e+00,14998,14839,270,210211-2-05,936.67426,27065.9940,Hyperglycemia,0.002308,395.6700,18,27,900-1000


In [83]:
small_pg = result_df.nsmallest(int(0.75 * len(result_df)), 'PageRank')

In [84]:
small_pg

,Closeness,Node_ID:ID,PageRank,Between,Louv,LabelProp,Leiden,SpecID,WaveNumber:float,Absorbance:float,Status:LABEL,Widths:float,Prominences:float,GridSlot_x,GridSlot_y,Bin
43720,0.255907,7517501,0.000000,0.000000,40051,1,87,210519-2-42,1895.44420,768.20282,Hyperglycemia,0.013047,83.99957,37,0,1800-1900
39869,0.270733,484813,0.008966,0.000000,21424,1,30,210114-2-34,1982.23230,951.05017,Normal,0.000567,76.09692,39,0,1900-2000
50433,0.208930,361035,0.020301,4442.528753,3526,311,126,210114-1-38,227.33485,896.20660,Normal,0.010738,113.27526,4,0,200-300
39189,0.273312,5607088,0.022686,6747.341325,17535,28,398,210414-3-34,1869.47610,1834.22610,Normal,0.000452,77.25060,37,1,1800-1900
42180,0.261305,57890,0.029254,4490.952140,46582,6,333,201210-1-22,1946.01370,1826.18520,Normal,0.000735,81.48000,38,1,1900-2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33278,0.286005,529758,0.719857,108122.066545,811,49502,104,210120-1-02,284.05466,2015.72110,Hyperglycemia,0.008252,106.93550,5,2,200-300
26837,0.293702,3869781,0.719861,29878.471769,40051,6,121,210324-2-19,1294.07740,1215.14640,Normal,0.029356,156.72980,25,1,1200-1300
28258,0.292389,1627595,0.719861,30398.775594,29762,6,84,210218-1-18,1430.06840,1428.27510,Hyperglycemia,0.032951,186.81450,28,1,1400-1500
31828,0.288100,2593183,0.719863,115471.347888,42349,4148,136,210304-2-35,434.39636,6647.06250,Hypoglycemia,0.003468,186.36670,8,6,400-500


In [94]:
# Pivot table with 'Absorbance', 'PeakWidths', and 'PeakProminences' as values
peak_bins = small_pg.pivot_table(index='SpecID', columns='Bin', values=['Widths:float', 'Prominences:float', 'GridSlot_x', 'GridSlot_y'], aggfunc='mean')
peak_bins.columns = [f"{col[0]}_{col[1]}" for col in peak_bins.columns]  # Combine column names
peak_bins.reset_index(inplace=True)

# Merge with 'Status' information
statuses = small_pg[['SpecID', 'Status:LABEL']].drop_duplicates()
peak_bins = pd.merge(peak_bins, statuses, on='SpecID')

# Set 'SpecID' as the index
peak_bins.set_index('SpecID', inplace=True)

# Fill NaN values with False
peak_bins.fillna(False, inplace=True)

C:\Users\stang\AppData\Local\Temp\ipykernel_13812\2536314543.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  peak_bins.fillna(False, inplace=True)


In [95]:
# Splitting the dataframe into features (X) and target variable (y)
X = peak_bins.drop(['Status:LABEL'], axis=1)
y = peak_bins['Status:LABEL']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [96]:
rf = RandomForestClassifier(random_state=1234)
rf.fit(X_train, y_train)

# Initialising and fitting the Extra Trees classifier
et = ExtraTreesClassifier(random_state=1234)
et.fit(X_train, y_train)

ExtraTreesClassifier(random_state=1234)

In [97]:
# Making predictions on the test set
y_pred = rf.predict(X_test)

# Calculating and printing evaluation metrics
calculate_metrics(y_test, y_pred)

Overall Accuracy: 0.7885906040268457


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.78      0.77      0.78       195
 Hypoglycemia       0.80      0.80      0.80       196
       Normal       0.79      0.80      0.79       205

     accuracy                           0.79       596
    macro avg       0.79      0.79      0.79       596
 weighted avg       0.79      0.79      0.79       596


Confusion Matrix:
[[151  20  24]
 [ 20 156  20]
 [ 22  20 163]]
